In [59]:
import pyspark
import random
import time
from pyspark.sql import functions as F

from pyspark.sql.functions import *

from pyspark.sql import SparkSession

### JOINS

In [2]:
data1 = [
    (1,"Alice",1,5000),\
    (2,"Bob", 1, 4000),\
    (3,"Cathy", 2, 3500),\
    (4,"David",4, 4500),\
    (5,"Eve", 3, 6000),\
    (6,"Frank",3, 7000),
]
schema1 = ["ID", "Name" ,"Department_ID", "Salary"]

empdf = spark.createDataFrame(data1 , schema=schema1)
empdf.show()

data2 = [(1,'IT'),(2,'HR'),(3,'Payroll'),(5,'Sales')]
schema2 = ["Department_ID" , "Department"]

depdf = spark.createDataFrame(data2 , schema=schema2)
depdf.show()

empdf.join(depdf, empdf.Department_ID == depdf.Department_ID, 'inner').show()
empdf.join(depdf, empdf.Department_ID == depdf.Department_ID, 'left').show()
empdf.join(depdf, empdf.Department_ID == depdf.Department_ID, 'right').show()
empdf.join(depdf, empdf.Department_ID == depdf.Department_ID, 'full').show()
empdf.join(depdf, empdf.Department_ID == depdf.Department_ID, 'leftsemi').show()
empdf.join(depdf, empdf.Department_ID == depdf.Department_ID, 'leftanti').show()




C:\Users\ranju\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


+---+-----+-------------+------+
| ID| Name|Department_ID|Salary|
+---+-----+-------------+------+
|  1|Alice|            1|  5000|
|  2|  Bob|            1|  4000|
|  3|Cathy|            2|  3500|
|  4|David|            4|  4500|
|  5|  Eve|            3|  6000|
|  6|Frank|            3|  7000|
+---+-----+-------------+------+

+-------------+----------+
|Department_ID|Department|
+-------------+----------+
|            1|        IT|
|            2|        HR|
|            3|   Payroll|
|            5|     Sales|
+-------------+----------+

+---+-----+-------------+------+-------------+----------+
| ID| Name|Department_ID|Salary|Department_ID|Department|
+---+-----+-------------+------+-------------+----------+
|  1|Alice|            1|  5000|            1|        IT|
|  2|  Bob|            1|  4000|            1|        IT|
|  3|Cathy|            2|  3500|            2|        HR|
|  5|  Eve|            3|  6000|            3|   Payroll|
|  6|Frank|            3|  7000|            3|

In [9]:
help(empdf.join)

Help on method join in module pyspark.sql.dataframe:

join(other: 'DataFrame', on: Union[str, List[str], pyspark.sql.column.Column, List[pyspark.sql.column.Column], NoneType] = None, how: Optional[str] = None) -> 'DataFrame' method of pyspark.sql.dataframe.DataFrame instance
    Joins with another :class:`DataFrame`, using the given join expression.
    
    .. versionadded:: 1.3.0
    
    .. versionchanged:: 3.4.0
        Supports Spark Connect.
    
    Parameters
    ----------
    other : :class:`DataFrame`
        Right side of the join
    on : str, list or :class:`Column`, optional
        a string for the join column name, a list of column names,
        a join expression (Column), or a list of Columns.
        If `on` is a string or a list of strings indicating the name of the join column(s),
        the column(s) must exist on both sides, and this performs an equi-join.
    how : str, optional
        default ``inner``. Must be one of: ``inner``, ``cross``, ``outer``,
      

### Union and UnionAll

In [10]:
data1 = [(1,'bob','female'),(2,'Alice','male')]
schema1 = ['id','name','gender']

df1 = spark.createDataFrame(data1,schema=schema1)
df1.show()

data2 = [(1,'bob','female'),(3,'tom','male'),(4,'Eve','male')]
schema2 = ['id','name','gender']

df2 = spark.createDataFrame(data2,schema=schema2)
df2.show()

new_df = df1.union(df2).show()

new_df = df1.unionAll(df2)

new_df.distinct().show()


+---+-----+------+
| id| name|gender|
+---+-----+------+
|  1|  bob|female|
|  2|Alice|  male|
+---+-----+------+

+---+----+------+
| id|name|gender|
+---+----+------+
|  1| bob|female|
|  3| tom|  male|
|  4| Eve|  male|
+---+----+------+

+---+-----+------+
| id| name|gender|
+---+-----+------+
|  1|  bob|female|
|  2|Alice|  male|
|  1|  bob|female|
|  3|  tom|  male|
|  4|  Eve|  male|
+---+-----+------+

+---+-----+------+
| id| name|gender|
+---+-----+------+
|  1|  bob|female|
|  2|Alice|  male|
|  3|  tom|  male|
|  4|  Eve|  male|
+---+-----+------+



### UnionByName

In [15]:
data1 = [(1,'bob','female'),(2,'Alice','male')]
schema1 = ['id','name','gender']

df1 = spark.createDataFrame(data1,schema=schema1)
df1.show()

data2 = [(1,'bob',31),(3,'tom',22),(4,'Eve',25)]
schema2 = ['id','name','age']

df2 = spark.createDataFrame(data2,schema=schema2)
df2.show()

df1.union(df2).show()
df1.unionByName(df2,allowMissingColumns= True).show()

+---+-----+------+
| id| name|gender|
+---+-----+------+
|  1|  bob|female|
|  2|Alice|  male|
+---+-----+------+

+---+----+---+
| id|name|age|
+---+----+---+
|  1| bob| 31|
|  3| tom| 22|
|  4| Eve| 25|
+---+----+---+

+---+-----+------+
| id| name|gender|
+---+-----+------+
|  1|  bob|female|
|  2|Alice|  male|
|  1|  bob|    31|
|  3|  tom|    22|
|  4|  Eve|    25|
+---+-----+------+

+---+-----+------+----+
| id| name|gender| age|
+---+-----+------+----+
|  1|  bob|female|NULL|
|  2|Alice|  male|NULL|
|  1|  bob|  NULL|  31|
|  3|  tom|  NULL|  22|
|  4|  Eve|  NULL|  25|
+---+-----+------+----+



### Sample

In [23]:
df = spark.range(start=1, end =101)

df1 = df.sample(fraction=0.1, seed=123)
df1.show()


df2 = df.sample(fraction=0.1,seed=123)
df2.show()

+---+
| id|
+---+
| 27|
| 30|
| 59|
| 70|
| 92|
| 96|
| 99|
+---+

+---+
| id|
+---+
| 27|
| 30|
| 59|
| 70|
| 92|
| 96|
| 99|
+---+



In [25]:
spark.stop()

### User Defined Functions

In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [29]:
data = [
    (1,"Alice",80000,2000),\
    (2,"Bob", 100000, 4000),\
    (3,"Cathy", 90000, 3500) ]
schema = ["ID", "Name" , "Salary", "Bonus"]

df = spark.createDataFrame(data,schema)
df.show()


+---+-----+------+-----+
| ID| Name|Salary|Bonus|
+---+-----+------+-----+
|  1|Alice| 80000| 2000|
|  2|  Bob|100000| 4000|
|  3|Cathy| 90000| 3500|
+---+-----+------+-----+



In [30]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def total_sal(s,b):
    return s+b

Total = udf(lambda s,b :total_sal(s,b), returnType= IntegerType() ) #registering

In [31]:
df.withColumn("Total_payment", Total(df.Salary,df.Bonus)).show()

+---+-----+------+-----+-------------+
| ID| Name|Salary|Bonus|Total_payment|
+---+-----+------+-----+-------------+
|  1|Alice| 80000| 2000|        82000|
|  2|  Bob|100000| 4000|       104000|
|  3|Cathy| 90000| 3500|        93500|
+---+-----+------+-----+-------------+



In [34]:
# another way for registering


@udf(returnType=IntegerType())
def total_sal(s,b):
    return s+b

df.select('*' , total_sal(df.Salary,df.Bonus).alias("Total_payment")).show()

+---+-----+------+-----+-------------+
| ID| Name|Salary|Bonus|Total_payment|
+---+-----+------+-----+-------------+
|  1|Alice| 80000| 2000|        82000|
|  2|  Bob|100000| 4000|       104000|
|  3|Cathy| 90000| 3500|        93500|
+---+-----+------+-----+-------------+



In [54]:
data = [
    (1,"Alice",80000,2000),\
    (2,"Bob", 100000, 4000),\
    (3,"Cathy", 90000, 3500) ]
schema = ["ID", "Name" , "Salary", "Bonus"]

df = spark.createDataFrame(data,schema)

df.createOrReplaceTempView('df')

def total_sal(s,b):
    return s+b

spark.udf.register(name = 'total' ,f =total_sal, returnType=IntegerType()) #registering 

result = spark.sql('SELECT *, total(Salary, Bonus) AS total_payment FROM df')
result.show()

+---+-----+------+-----+-------------+
| ID| Name|Salary|Bonus|total_payment|
+---+-----+------+-----+-------------+
|  1|Alice| 80000| 2000|        82000|
|  2|  Bob|100000| 4000|       104000|
|  3|Cathy| 90000| 3500|        93500|
+---+-----+------+-----+-------------+



### Example on UDF

In [94]:
data = [("Table", 34), ("Chair", 18), ("Lamp", 25), ("Couch", 120)]
schema = ["Category", "Price"]

df = spark.createDataFrame(data, schema)
df.show()

+--------+-----+
|Category|Price|
+--------+-----+
|   Table|   34|
|   Chair|   18|
|    Lamp|   25|
|   Couch|  120|
+--------+-----+



In [98]:
from pyspark.sql.functions import StringType

@udf(returnType=StringType())
def categorize_price(price):
    if price < 30:
        return "Low"
    elif price < 100:
        return "Medium"
    else:
        return "High"


df.select('*' , categorize_price(df.Price).alias('price_Category')).s+how()


+--------+-----+--------------+
|Category|Price|price_Category|
+--------+-----+--------------+
|   Table|   34|        Medium|
|   Chair|   18|           Low|
|    Lamp|   25|           Low|
|   Couch|  120|          High|
+--------+-----+--------------+



### Example on Sample

In [89]:
df = spark.read.csv(r"C:\Users\ranju\Downloads\large_sales_data.csv",inferSchema= True ,header=True)
df.show()

+-------+---------+-----------+-----+--------+------+----------+----------+
|OrderID|  Product|   Category|Price|Quantity|Region|      Date|Total_Sale|
+-------+---------+-----------+-----+--------+------+----------+----------+
|   1001|    Table|Electronics|   34|       7|  East|2024-12-01|       238|
|   1002|     Desk|Electronics|  832|       7| South|2024-12-02|      5824|
|   1003|Headphone|  Furniture|   84|       9|  East|2024-12-03|       756|
|   1004|    Phone|Electronics|  876|       3| North|2024-12-04|      2628|
|   1005|    Table|  Furniture|  858|       7|  East|2024-12-05|      6006|
|   1006|     Lamp|  Furniture|  540|       1|  West|2024-12-06|       540|
|   1007|    Chair|  Furniture|  363|       4|  West|2024-12-07|      1452|
|   1008|    Table|  Furniture|  148|       4| South|2024-12-08|       592|
|   1009|Headphone|  Furniture|  667|       5|  West|2024-12-09|      3335|
|   1010|    Phone|  Furniture|  491|       7| North|2024-12-10|      3437|
|   1011|   

In [90]:
sample_df = df.sample(fraction=0.3 , seed =123)
sample_df.show()


+-------+---------+-----------+-----+--------+------+----------+----------+
|OrderID|  Product|   Category|Price|Quantity|Region|      Date|Total_Sale|
+-------+---------+-----------+-----+--------+------+----------+----------+
|   1001|    Table|Electronics|   34|       7|  East|2024-12-01|       238|
|   1005|    Table|  Furniture|  858|       7|  East|2024-12-05|      6006|
|   1018|Headphone|  Furniture|  308|       5|  East|2024-12-18|      1540|
|   1020|    Mouse|  Furniture|  792|       4|  West|2024-12-20|      3168|
|   1025|     Sofa|  Furniture|  154|       1| South|2024-12-25|       154|
|   1026|   Laptop|Electronics|  220|       9| North|2024-12-26|      1980|
|   1027|     Lamp|  Furniture|  859|       9| North|2024-12-27|      7731|
|   1037|     Sofa|  Furniture|  118|       3|  West|2025-01-06|       354|
|   1038|    Table|Electronics|  703|       8|  West|2025-01-07|      5624|
|   1042|    Mouse|  Furniture|  980|       4| South|2025-01-11|      3920|
|   1044|   

In [91]:
# Filter orders where the Category is 'Furniture' and Quantity > 4
filter_df = sample_df.filter((sample_df['Category'] == 'Furniture') & (sample_df['Quantity'] > 4))
filter_df.show()

+-------+---------+---------+-----+--------+------+----------+----------+
|OrderID|  Product| Category|Price|Quantity|Region|      Date|Total_Sale|
+-------+---------+---------+-----+--------+------+----------+----------+
|   1005|    Table|Furniture|  858|       7|  East|2024-12-05|      6006|
|   1018|Headphone|Furniture|  308|       5|  East|2024-12-18|      1540|
|   1027|     Lamp|Furniture|  859|       9| North|2024-12-27|      7731|
+-------+---------+---------+-----+--------+------+----------+----------+



In [92]:
# calculate total sales per region

r_df = sample_df.groupBy('Region').agg(F.sum('Total_Sale'))
r_df.show()


+------+---------------+
|Region|sum(Total_Sale)|
+------+---------------+
| South|           7922|
|  East|           8416|
|  West|           9146|
| North|           9711|
+------+---------------+



In [111]:
# Sort the data by Total_Sale in descending order

s_df = sample_df.orderBy(col('Total_Sale').desc())
s_df.show()

+-------+---------+-----------+-----+--------+------+----------+----------+
|OrderID|  Product|   Category|Price|Quantity|Region|      Date|Total_Sale|
+-------+---------+-----------+-----+--------+------+----------+----------+
|   1027|     Lamp|  Furniture|  859|       9| North|2024-12-27|      7731|
|   1005|    Table|  Furniture|  858|       7|  East|2024-12-05|      6006|
|   1038|    Table|Electronics|  703|       8|  West|2025-01-07|      5624|
|   1042|    Mouse|  Furniture|  980|       4| South|2025-01-11|      3920|
|   1045|     Lamp|  Furniture|  962|       4| South|2025-01-14|      3848|
|   1020|    Mouse|  Furniture|  792|       4|  West|2024-12-20|      3168|
|   1026|   Laptop|Electronics|  220|       9| North|2024-12-26|      1980|
|   1018|Headphone|  Furniture|  308|       5|  East|2024-12-18|      1540|
|   1044|     Sofa|Electronics|  632|       1|  East|2025-01-13|       632|
|   1037|     Sofa|  Furniture|  118|       3|  West|2025-01-06|       354|
|   1001|   

In [112]:
spark.stop()